In [34]:
%pip install nltk
%pip install numpy
%pip install keras
%pip install tensorflow
%pip install gtts
%pip install playsound 
%pip install pygame

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\foold\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\foold\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


...

You should consider upgrading via the 'c:\Users\foold\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\foold\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\foold\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [35]:
import nltk
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from gtts import gTTS 
from pygame import mixer

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\foold\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\foold\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\foold\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [36]:
### CREATE VIRTUAL DISPLAY ###
!apt-get install -y xvfb # Install X Virtual Frame Buffer
import os
os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
os.environ['DISPLAY']=':1.0'    # tell X clients to use our virtual DISPLAY :1.0.

'apt-get' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [37]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('C:\\Users\\foold\\Downloads\\intents.json').read()
intents = json.loads(data_file)

In [38]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [39]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

127 documents
31 classes ['Deus', 'aberto hoje', 'adeus', 'adverse_drug', 'aluguel', 'blood_pressure_search', 'brincadeira', 'criador', 'eu mesmo', 'falar', 'google', 'histÃ³rias', 'hoje', 'horas', 'hospital_search', 'matar', 'motocicletas', 'notÃ\xadcias', 'obrigado', 'opÃ§Ãµes', 'pagamentos', 'pharmacy_search', 'pressÃ£o_sangue', 'reservas', 'saudaÃ§Ã£o', 'sexta-feira', 'sobre mim', 'tarefas', 'tempo', 'vivo', 'wikipÃ©dia']
185 unique lemmatized words [',', 'a', 'aberto', 'abra', 'abre', 'abrir', 'aceita', 'adequados', 'adeus', 'adversa', 'adverso', 'adversos', 'ai', 'ajuda', 'ajudar', 'algo', 'alguma', 'alguã©m', 'aluga', 'alugar', 'apenas', 'arterial', 'atuais', 'atualizaã§ãµes', 'atã©', 'aã\xad', 'bilhete', 'bom', 'cantar', 'caracterã\xadsticas', 'carregar', 'cartãµes', 'causam', 'ciclomotor', 'ciclomotores', 'cientã\xadfica', 'clima', 'coisa', 'com', 'como', 'comportamento', 'conhece', 'contar', 'conte', 'conversar', 'correr', 'criador', 'criou', 'crã©dito', 'dado', 'de', 'detalh

In [40]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [41]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=90, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

c:\Users\foold\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/90
26/26 [==============================] - 0s 801us/step - loss: 3.4224 - accuracy: 0.0630
Epoch 2/90
26/26 [==============================] - 0s 841us/step - loss: 3.3444 - accuracy: 0.0787
Epoch 3/90
26/26 [==============================] - 0s 801us/step - loss: 3.2653 - accuracy: 0.1417
Epoch 4/90
26/26 [==============================] - 0s 801us/step - loss: 3.2022 - accuracy: 0.1417
Epoch 5/90
26/26 [==============================] - 0s 841us/step - loss: 3.0628 - accuracy: 0.1969
Epoch 6/90
26/26 [==============================] - 0s 761us/step - loss: 2.9542 - accuracy: 0.2126
Epoch 7/90
26/26 [==============================] - 0s 801us/step - loss: 2.7196 - accuracy: 0.2756
Epoch 8/90
26/26 [==============================] - 0s 761us/step - loss: 2.6800 - accuracy: 0.2913
Epoch 9/90
26/26 [==============================] - 0s 761us/step - loss: 2.4455 - accuracy: 0.3937
Epoch 10/90
26/26 [==============================] - 0s 801us/step - loss: 2.3742 - accuracy: 0.3858

In [42]:
from keras.models import load_model
model = load_model('C:\\Users\\foold\\Downloads\\Voicebot-Project-main\\Voicebot-Project-main\\voicebot\\chatbot_model.h5')
import json
import random
intents = json.loads(open('C:\\Users\\foold\\Downloads\\intents.json').read())
words = pickle.load(open('C:\\Users\\foold\\Downloads\\Voicebot-Project-main\\Voicebot-Project-main\\voicebot\\words.pkl','rb'))
classes = pickle.load(open('C:\\Users\\foold\\Downloads\\Voicebot-Project-main\\Voicebot-Project-main\\voicebot\\classes.pkl','rb'))

In [43]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [44]:
#Creating GUI with tkinter
import tkinter
from tkinter import *
import time

def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
    
    # The text that you want to convert to audio
    mytext = res
        
    # Language in which you want to convert
    language = 'pt'
        
    # Passing the text and language to the engine, 
    # here we have marked slow=False. Which tells 
    # the module that the converted audio should 
    # have a high speed
    myobj = gTTS(text=mytext, lang=language, slow=False)
        
    # Saving the converted audio in a mp3 file named
    # welcome 
    myobj.save("welcome.mp3")


mixer.init()
mixer.music.unload()
def play():
    mixer.music.load("welcome.mp3")
    mixer.music.play()
    if os.path.exists('welcome.mp3'):
        while mixer.music.get_busy():
            pass
        mixer.music.unload()
        os.remove('welcome.mp3')
        

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command=lambda: [send(), play()])
    

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

1/1 [==============================] - 0s 38ms/step
